In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
graf_raw = pd.read_csv("data/311_Service_Requests_-_Graffiti_Removal.csv")
build_raw = pd.read_csv("data/311_Service_Requests_-_Vacant_and_Abandoned_Buildings_Reported.csv")
light_raw = pd.read_csv("data/311_Service_Requests_-_Alley_Lights_Out.csv")
commarea_raw = pd.read_csv("data/CommAreas.csv")

In [7]:
graf_new = graf_raw[graf_raw['Status'].isin(['Completed', 'Open'])]
light_new = light_raw[light_raw['Status'].isin(['Completed', 'Open'])]
build_new = build_raw.dropna(subset=['ADDRESS STREET NUMBER', \
                                 'ADDRESS STREET DIRECTION',\
                                 'ADDRESS STREET NAME', \
                                 'ADDRESS STREET SUFFIX'])

In [11]:
graf = graf_new[['Service Request Number', 'Type of Service Request',\
                 'Street Address','Community Area']]
light = light_new[['Service Request Number', 'Type of Service Request',\
                 'Street Address','Community Area']]

In [20]:
build_broken_address = build_new[['ADDRESS STREET NUMBER', 'ADDRESS STREET DIRECTION','ADDRESS STREET NAME', 'ADDRESS STREET SUFFIX']]
seam = build_broken_address.astype(str).apply(lambda x: ' '.join(x), axis=1)
build_select = pd.concat([build_new, seam], axis=1)
build_select.rename(columns={0:'Street Address'},inplace=True)
build = build_select[['SERVICE REQUEST NUMBER', 'SERVICE REQUEST TYPE',\
                  'Street Address', 'Community Area']]

In [101]:
build.columns = graf.columns
light.columns = graf.columns
combinedf = pd.concat([graf, build, light], ignore_index=True)

In [102]:
commarea = commarea_raw[['AREA_NUMBE', 'COMMUNITY']].rename(columns={'AREA_NUMBE':'Community Area', \
                                                                     'COMMUNITY':'Neighborhood'})
combinedf_area = pd.merge(combinedf, commarea, how='right', on='Community Area')

In [103]:
st = combinedf_area[combinedf_area['Street Address'].str.contains('[2-4][0-9]{3} W ROOSEVELT')]
type_on_st = st.groupby('Type of Service Request').size()/len(st)
type_on_st

Type of Service Request
Alley Light Out              0.150538
Graffiti Removal             0.838710
Vacant/Abandoned Building    0.010753
dtype: float64

In [104]:
temp = combinedf_area[combinedf_area['Type of Service Request']=='Graffiti Removal']
graffiti_by_area = temp.groupby('Neighborhood').size()/len(temp)

In [107]:
graffiti_by_area['UPTOWN']

0.016172805317086678

In [108]:
graffiti_by_area['GARFIELD RIDGE']

0.006997138373488415